<a href="https://colab.research.google.com/github/Stekosan/merrer/blob/main/Copy_of_SparkNLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Let's set up SparkNLP.

In [1]:
!wget http://setup.johnsnowlabs.com/colab.sh -O - | bash

--2024-07-27 06:36:01--  http://setup.johnsnowlabs.com/colab.sh
Resolving setup.johnsnowlabs.com (setup.johnsnowlabs.com)... 3.86.22.73
Connecting to setup.johnsnowlabs.com (setup.johnsnowlabs.com)|3.86.22.73|:80... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp/master/scripts/colab_setup.sh [following]
--2024-07-27 06:36:01--  https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp/master/scripts/colab_setup.sh
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1191 (1.2K) [text/plain]
Saving to: ‘STDOUT’

-                   100%[===================>]   1.16K  --.-KB/s    in 0s      

2024-07-27 06:36:01 (50.4 MB/s) - written to stdout [1191/1191]

Installing PySpark

In [2]:
import sparknlp
spark = sparknlp.start()

from sparknlp.pretrained import PretrainedPipeline

In [3]:
pipeline = PretrainedPipeline("explain_document_ml")

explain_document_ml download started this may take some time.
Approx size to download 9 MB
[OK!]


We can use some recent headlines.

In [4]:
hls = [ # was headlines
		"She",
		"He",
		"her",
		"him",
		"hers",
		"his"
	]

Let's use SparkNLP to analyze these headlines.

In [5]:
# Use dataframes, or...
# data = spark.createDataFrame(hls).toDF("text")
# dfs = pipeline.transform(data)
# ... use list comprehension
dfs = [pipeline.annotate(hl) for hl in hls] # I don't know how to use dataframes

In [6]:
# its big
dfs

[{'document': ['She'],
  'spell': ['She'],
  'pos': ['PRP'],
  'lemmas': ['She'],
  'token': ['She'],
  'stems': ['she'],
  'sentence': ['She']},
 {'document': ['He'],
  'spell': ['He'],
  'pos': ['PRP'],
  'lemmas': ['He'],
  'token': ['He'],
  'stems': ['he'],
  'sentence': ['He']},
 {'document': ['her'],
  'spell': ['her'],
  'pos': ['PRP$'],
  'lemmas': ['she'],
  'token': ['her'],
  'stems': ['her'],
  'sentence': ['her']},
 {'document': ['him'],
  'spell': ['him'],
  'pos': ['PRP'],
  'lemmas': ['he'],
  'token': ['him'],
  'stems': ['him'],
  'sentence': ['him']},
 {'document': ['hers'],
  'spell': ['hers'],
  'pos': ['NNS'],
  'lemmas': ['hers'],
  'token': ['hers'],
  'stems': ['her'],
  'sentence': ['hers']},
 {'document': ['his'],
  'spell': ['his'],
  'pos': ['PRP$'],
  'lemmas': ['he'],
  'token': ['his'],
  'stems': ['hi'],
  'sentence': ['his']}]

Let's say we want to fuse part-of-speech tags to words, to make word differentiation easier.

In [7]:
# Extract words and parts-of-speech
tok_tag = [(df['token'],df['pos']) for df in dfs]

In [8]:
# Still big
tok_tag

[(['She'], ['PRP']),
 (['He'], ['PRP']),
 (['her'], ['PRP$']),
 (['him'], ['PRP']),
 (['hers'], ['NNS']),
 (['his'], ['PRP$'])]

In [9]:
# fuse pos to word
zips = [list(zip(tt[0], tt[1])) for tt in tok_tag]

In [10]:
# not too big
zips

[[('She', 'PRP')],
 [('He', 'PRP')],
 [('her', 'PRP$')],
 [('him', 'PRP')],
 [('hers', 'NNS')],
 [('his', 'PRP$')]]

In [11]:
tagged = [" ".join(["".join(word) for word in hl]) for hl in zips]

In [12]:
tagged

['ShePRP', 'HePRP', 'herPRP$', 'himPRP', 'hersNNS', 'hisPRP$']

What about ebooks?

In [13]:
!curl "https://raw.githubusercontent.com/cd-public/books/main/pg1342.txt" -o austen.txt

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  739k  100  739k    0     0  2135k      0 --:--:-- --:--:-- --:--:-- 2138k


In [14]:
austen = open('austen.txt').read()

In [15]:
print(austen[:1000])

﻿The Project Gutenberg eBook of Pride and Prejudice
    
This ebook is for the use of anyone anywhere in the United States and
most other parts of the world at no cost and with almost no restrictions
whatsoever. You may copy it, give it away or re-use it under the terms
of the Project Gutenberg License included with this ebook or online
at www.gutenberg.org. If you are not located in the United States,
you will have to check the laws of the country where you are located
before using this eBook.

Title: Pride and Prejudice

Author: Jane Austen

Release date: June 1, 1998 [eBook #1342]
                Most recently updated: April 14, 2023

Language: English

Credits: Chuck Greif and the Online Distributed Proofreading Team at http://www.pgdp.net (This file was produced from images available at The Internet Archive)


*** START OF THE PROJECT GUTENBERG EBOOK PRIDE AND PREJUDICE ***





                            [Illustration:

                             GEORGE ALLEN
                 

In [16]:
pipeline.annotate(austen[:100])['pos']

['DT',
 'NNP',
 'NNP',
 'NN',
 'IN',
 'NNP',
 'CC',
 'NNP',
 'DT',
 'NN',
 'VBZ',
 'IN',
 'DT',
 'NN',
 'IN',
 'NN',
 'RB']

Previously with ebooks, we conducted word counts. We can do that here as well, with Spark.

In [17]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("demo").getOrCreate()

In [18]:
# change 'austen' variable from a string to a spark object
austen = spark.sparkContext.textFile("austen.txt")

counts = (
    austen.flatMap(lambda line: line.split(" "))
    .map(lambda word: (word, 1))
    .reduceByKey(lambda a, b: a + b)
)

In [19]:
counts.collect()[:10]

[('The', 285),
 ('Project', 79),
 ('of', 3897),
 ('Pride', 7),
 ('', 10603),
 ('ebook', 2),
 ('is', 861),
 ('use', 23),
 ('anyone', 20),
 ('anywhere', 3)]